In [82]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric

# Load the data
train = pd.read_csv(r'../Resources/Data/baseline_train_data.csv')
test = pd.read_csv(r'../Resources/Data/baseline_test_data.csv')
valid = pd.read_csv(r'../Resources/Data/baseline_valis_data.csv')

# Define target and features
target = 'two_year_recid'

# Separate features and target for training, testing, and validation data
X_train = train.drop(columns=[target])
y_train = train[target]

X_test = test.drop(columns=[target])
y_test = test[target]

X_valid = valid.drop(columns=[target])
y_valid = valid[target]

# Fit the logistic regression model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# Predict on the test data
y_pred_test = log_reg.predict(X_test)

# Combine predicted labels with the test features
test_with_pred = X_test.copy()
test_with_pred[target] = y_test
test_with_pred['y_pred'] = y_pred_test

# Convert to AIF360 BinaryLabelDataset format
test_aif = BinaryLabelDataset(df=pd.concat([X_test, y_test], axis=1),
                              label_names=[target],
                              protected_attribute_names=['sex'])

test_aif_pred = BinaryLabelDataset(df=pd.concat([X_test, pd.Series(y_pred_test, name=target)], axis=1),
                                   label_names=[target],
                                   protected_attribute_names=['sex'])

# Define privileged and unprivileged groups
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

# Compute fairness metrics
metric_test_pred = ClassificationMetric(test_aif, test_aif_pred,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)

stat_parity_diff = metric_test_pred.statistical_parity_difference()
avg_odds_diff = metric_test_pred.average_odds_difference()
equal_opp_diff = metric_test_pred.equal_opportunity_difference()

# Display results
print(f"Statistical Parity Difference: {stat_parity_diff}")
print(f"Average Odds Difference: {avg_odds_diff}")
print(f"Equal Opportunity Difference: {equal_opp_diff}")


Statistical Parity Difference: 0.23776896724277302
Average Odds Difference: 0.22409495346851444
Equal Opportunity Difference: 0.22007930785868784


In [84]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.metrics import ClassificationMetric

# Load the data
train = pd.read_csv(r'../Resources/Data/baseline_train_data.csv')
test = pd.read_csv(r'../Resources/Data/baseline_test_data.csv')
valid = pd.read_csv(r'../Resources/Data/baseline_valis_data.csv')

# Define target and features
target = 'two_year_recid'

# Separate features and target for training, testing, and validation data
X_train = train.drop(columns=[target])
y_train = train[target]

X_test = test.drop(columns=[target])
y_test = test[target]

X_valid = valid.drop(columns=[target])
y_valid = valid[target]

# Convert training data to AIF360 BinaryLabelDataset format
train_aif = BinaryLabelDataset(df=pd.concat([X_train, y_train], axis=1),
                               label_names=[target],
                               protected_attribute_names=['sex'])

# Apply reweighting to the training set
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

reweigher = Reweighing(unprivileged_groups=unprivileged_groups,
                       privileged_groups=privileged_groups)
train_aif_rw = reweigher.fit_transform(train_aif)

# Fit the logistic regression model with sample weights
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(train_aif_rw.features, train_aif_rw.labels.ravel(), sample_weight=train_aif_rw.instance_weights)

# Predict on the test data
y_pred_test = log_reg.predict(X_test)

# Combine predicted labels with the test features
test_with_pred = X_test.copy()
test_with_pred[target] = y_test
test_with_pred['y_pred'] = y_pred_test

# Convert to AIF360 BinaryLabelDataset format
test_aif = BinaryLabelDataset(df=pd.concat([X_test, y_test], axis=1),
                              label_names=[target],
                              protected_attribute_names=['sex'])

test_aif_pred = BinaryLabelDataset(df=pd.concat([X_test, pd.Series(y_pred_test, name=target)], axis=1),
                                   label_names=[target],
                                   protected_attribute_names=['sex'])

# Compute fairness metrics
metric_test_pred = ClassificationMetric(test_aif, test_aif_pred,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)

stat_parity_diff = metric_test_pred.statistical_parity_difference()
avg_odds_diff = metric_test_pred.average_odds_difference()
equal_opp_diff = metric_test_pred.equal_opportunity_difference()

# Display results
print(f"Statistical Parity Difference: {stat_parity_diff}")
print(f"Average Odds Difference: {avg_odds_diff}")
print(f"Equal Opportunity Difference: {equal_opp_diff}")


Statistical Parity Difference: 0.05941337233919708
Average Odds Difference: 0.049523719720250076
Equal Opportunity Difference: 0.08525594808940151


C:\Users\General\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [97]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric

# Load the data
train = pd.read_csv(r'../Resources/Data/baseline_train_data.csv')
test = pd.read_csv(r'../Resources/Data/baseline_test_data.csv')
valid = pd.read_csv(r'../Resources/Data/baseline_valis_data.csv')

# Define target and features
target = 'two_year_recid'

# Separate features and target for training, testing, and validation data
X_train = train.drop(columns=[target])
y_train = train[target]

X_test = test.drop(columns=[target])
y_test = test[target]

X_valid = valid.drop(columns=[target])
y_valid = valid[target]

# Define sampling strategies
over_sampler = SMOTE(random_state=42)
under_sampler = RandomUnderSampler(random_state=42)
combined_sampler = SMOTEENN(random_state=42)

# Function to train and evaluate the model with different sampling techniques
def evaluate_fairness(X_train, y_train, sampling_strategy=None):
    if sampling_strategy:
        X_train, y_train = sampling_strategy.fit_resample(X_train, y_train)
    
    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(X_train, y_train)
    
    # Predict on the test data
    y_pred_test = log_reg.predict(X_test)
    
    # Convert to AIF360 BinaryLabelDataset format
    test_aif = BinaryLabelDataset(df=pd.concat([X_test, y_test], axis=1),
                                  label_names=[target],
                                  protected_attribute_names=['sex'])
    
    test_aif_pred = BinaryLabelDataset(df=pd.concat([X_test, pd.Series(y_pred_test, name=target)], axis=1),
                                       label_names=[target],
                                       protected_attribute_names=['sex'])
    
    # Define privileged and unprivileged groups
    privileged_groups = [{'sex': 1}]
    unprivileged_groups = [{'sex': 0}]
    
    # Compute fairness metrics
    metric_test_pred = ClassificationMetric(test_aif, test_aif_pred,
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
    
    stat_parity_diff = metric_test_pred.statistical_parity_difference()
    avg_odds_diff = metric_test_pred.average_odds_difference()
    equal_opp_diff = metric_test_pred.equal_opportunity_difference()
    
    # Display results
    print(f"Sampling Strategy: {sampling_strategy.__class__.__name__ if sampling_strategy else 'None'}")
    print(f"Statistical Parity Difference: {stat_parity_diff}")
    print(f"Average Odds Difference: {avg_odds_diff}")
    print(f"Equal Opportunity Difference: {equal_opp_diff}")
    print()

# Evaluate the model with different sampling strategies
print("With Over-Sampling (SMOTE):")
evaluate_fairness(X_train, y_train, over_sampler)

print("With Under-Sampling (RandomUnderSampler):")
evaluate_fairness(X_train, y_train, under_sampler)

print("With Combined Sampling (SMOTE + ENN):")
evaluate_fairness(X_train, y_train, combined_sampler)


With Over-Sampling (SMOTE):
Sampling Strategy: SMOTE
Statistical Parity Difference: 0.32795719619692826
Average Odds Difference: 0.315755227048678
Equal Opportunity Difference: 0.3286319394376352

With Under-Sampling (RandomUnderSampler):
Sampling Strategy: RandomUnderSampler
Statistical Parity Difference: 0.3423341930020401
Average Odds Difference: 0.3300062077613293
Equal Opportunity Difference: 0.3385003604902668

With Combined Sampling (SMOTE + ENN):
Sampling Strategy: SMOTEENN
Statistical Parity Difference: 0.25447476808191233
Average Odds Difference: 0.24212005215492188
Equal Opportunity Difference: 0.24968457101658253



In [115]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.metrics import ClassificationMetric

# Load the data
train = pd.read_csv(r'../Resources/Data/baseline_train_data.csv')
test = pd.read_csv(r'../Resources/Data/baseline_test_data.csv')
valid = pd.read_csv(r'../Resources/Data/baseline_valis_data.csv')

# Define target and features
target = 'two_year_recid'

# Separate features and target for training, testing, and validation data
X_train = train.drop(columns=[target])
y_train = train[target]

X_test = test.drop(columns=[target])
y_test = test[target]

X_valid = valid.drop(columns=[target])
y_valid = valid[target]

# Convert to AIF360 BinaryLabelDataset format
train_aif = BinaryLabelDataset(df=pd.concat([X_train, y_train], axis=1),
                                label_names=[target],
                                protected_attribute_names=['sex'])

test_aif = BinaryLabelDataset(df=pd.concat([X_test, y_test], axis=1),
                               label_names=[target],
                               protected_attribute_names=['sex'])

# Apply Disparate Impact Remover
dir = DisparateImpactRemover(repair_level=1.0)
train_aif_transf = dir.fit_transform(train_aif)

# Extract the transformed features and target
X_train_transf = train_aif_transf.features
y_train_transf = train_aif_transf.labels

# Train the logistic regression model on the transformed data
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_transf, y_train_transf)

# Predict on the test data
y_pred_test = log_reg.predict(X_test)

# Convert to AIF360 BinaryLabelDataset format for test set with predictions
test_aif_pred = BinaryLabelDataset(df=pd.concat([X_test, pd.Series(y_pred_test, name=target)], axis=1),
                                   label_names=[target],
                                   protected_attribute_names=['sex'])

# Define privileged and unprivileged groups
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

# Compute fairness metrics
metric_test_pred = ClassificationMetric(test_aif, test_aif_pred,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)

stat_parity_diff = metric_test_pred.statistical_parity_difference()
avg_odds_diff = metric_test_pred.average_odds_difference()
equal_opp_diff = metric_test_pred.equal_opportunity_difference()

# Display results
print("Disparate Impact Remover")
print(f"Statistical Parity Difference: {stat_parity_diff}")
print(f"Average Odds Difference: {avg_odds_diff}")
print(f"Equal Opportunity Difference: {equal_opp_diff}")
print()


Disparate Impact Remover
Statistical Parity Difference: 0.23776896724277302
Average Odds Difference: 0.22409495346851444
Equal Opportunity Difference: 0.22007930785868784



C:\Users\General\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\General\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
